# Machine Learning Models For Solar Cost Datasets

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
import pandas as pd
import tensorflow as tf
import numpy as np

# import xgboost dependencies
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance

# disable the gpu on mac_m2 so it runs correctly
# https://github.com/keras-team/tf-keras/issues/140
# https://github.com/tensorflow/tensorflow/issues/62361
hw = tf.config.get_visible_devices()
tf.config.set_visible_devices(hw[0])

In [2]:
# import the Utility dataset
utility = 'SDGE'
#data_file = "clean_data/df_" + utility + ".csv"
data_file = "/Users/ns96/Documents/ML_Project/df_" + utility + ".csv"

utility_df = pd.read_csv(data_file)
print(utility_df.shape)
utility_df.head()

(85620, 19)


,Utility,Service_City,Service_Zip,Service_County,Technology_Type,System_Size_AC,Storage_Size_kW_AC,Inverter_Size_kW_AC,Mounting_Method,App_Received_Date,Installer_Name,Third_Party_Owned,Electric_Vehicle,Total_System_Cost,Generator_Manufacturer,Inverter_Manufacturer,Generator_Quantity,Inverter_Quantity,Year
0,SDGE,SAN DIEGO,92105,SAN DIEGO,Solar,3.676,0.0,3.800,Rooftop,2019-02-14,Tesla,No,No,21709.0,Kyocera,ABB,16.0,1.0,2019
1,SDGE,SAN DIEGO,92110,SAN DIEGO,Solar,7.639,0.0,5.052,Rooftop,2019-09-24,Self-installed,No,No,38000.0,Hanwha,SolarEdge,24.0,1.0,2019
2,SDGE,LEMON GROVE,91945,SAN DIEGO,Solar,4.760,0.0,15.000,Rooftop,2018-11-30,Self-installed,No,No,18000.0,Other,SunPower,24.0,12.0,2018
3,SDGE,LA MESA,91941,SAN DIEGO,Solar,6.776,0.0,7.616,Rooftop,2018-01-17,Self-installed,No,No,22000.0,LG,SolarEdge,22.0,1.0,2018
4,SDGE,SAN DIEGO,92130,SAN DIEGO,Solar,4.320,0.0,3.747,Rooftop,2018-01-26,Semper,No,No,15962.0,Sanyo,SolarEdge,28.0,1.0,2018


In [3]:
# lets see what values of total cost are
utility_df.Total_System_Cost.describe()

count    8.562000e+04
mean     2.956006e+04
std      2.708759e+04
min      7.000000e+03
25%      1.718300e+04
50%      2.476300e+04
75%      3.570000e+04
max      2.976760e+06
Name: Total_System_Cost, dtype: float64

In [ ]:
# reduce the number of cities i.e if less than 200 then just set it to other
city_counts = utility_df.Service_City.value_counts().to_dict()

def check_count(city):
    if city_counts[city] >= 200:
        return city
    else:
        return 'Other'

utility_df['Service_City'] = utility_df['Service_City'].apply(check_count)

In [ ]:
utility_df['Service_City'].value_counts()

In [4]:
# convert zip code to string
utility_df['Service_Zip'] = utility_df['Service_Zip'].astype(int).astype(str).str.zfill(5)
utility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85620 entries, 0 to 85619
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Utility                 85620 non-null  object 
 1   Service_City            85620 non-null  object 
 2   Service_Zip             85620 non-null  object 
 3   Service_County          85620 non-null  object 
 4   Technology_Type         85620 non-null  object 
 5   System_Size_AC          85620 non-null  float64
 6   Storage_Size_kW_AC      85620 non-null  float64
 7   Inverter_Size_kW_AC     85620 non-null  float64
 8   Mounting_Method         85620 non-null  object 
 9   App_Received_Date       85620 non-null  object 
 10  Installer_Name          85620 non-null  object 
 11  Third_Party_Owned       85620 non-null  object 
 12  Electric_Vehicle        85620 non-null  object 
 13  Total_System_Cost       85620 non-null  float64
 14  Generator_Manufacturer  85620 non-null

In [5]:
# see how many installers, cities, and zip codes we have
print("Zipcodes:", utility_df.Service_Zip.nunique())
print("Cities:", utility_df.Service_City.nunique())
print("Installers:", utility_df.Installer_Name.nunique())
print("Generator Manufacturer:", utility_df.Generator_Manufacturer.nunique())

Zipcodes: 115
Cities: 66
Installers: 44
Generator Manufacturer: 31


In [6]:
#for now lest drop some columns to make model perform better?
#columns_to_drop = ['Utility', 'Service_Zip', 'Service_County', 'App_Received_Date', 'Inverter_Size_kW_AC', 
#                   'Generator_Manufacturer', 'Inverter_Manufacturer', 'Inverter_Quantity', 'Year']

# Used when training on the entire dataset
columns_to_drop = ['Service_Zip', 'Service_County', 'App_Received_Date', 'Inverter_Size_kW_AC', 
                   'Inverter_Manufacturer', 'Inverter_Quantity', 'Year']

#columns_to_drop = ['Utility', 'Service_Zip', 'Service_County', 'App_Received_Date', 'Inverter_Size_kW_AC', 
#                   'Inverter_Manufacturer', 'Inverter_Quantity', 'Year']

#columns_to_drop = ['Utility', 'Service_Zip', 'App_Received_Date']
#columns_to_drop = ['Utility', 'Service_City', 'App_Received_Date']
#columns_to_drop = ['Utility', 'App_Received_Date']
#columns_to_drop = ['Utility', 'Service_Zip', 'App_Received_Date', 'Inverter_Size_kW_AC']

utility_df.drop(columns=columns_to_drop, inplace=True)
utility_df.head()

,Utility,Service_City,Technology_Type,System_Size_AC,Storage_Size_kW_AC,Mounting_Method,Installer_Name,Third_Party_Owned,Electric_Vehicle,Total_System_Cost,Generator_Manufacturer,Generator_Quantity
0,SDGE,SAN DIEGO,Solar,3.676,0.0,Rooftop,Tesla,No,No,21709.0,Kyocera,16.0
1,SDGE,SAN DIEGO,Solar,7.639,0.0,Rooftop,Self-installed,No,No,38000.0,Hanwha,24.0
2,SDGE,LEMON GROVE,Solar,4.760,0.0,Rooftop,Self-installed,No,No,18000.0,Other,24.0
3,SDGE,LA MESA,Solar,6.776,0.0,Rooftop,Self-installed,No,No,22000.0,LG,22.0
4,SDGE,SAN DIEGO,Solar,4.320,0.0,Rooftop,Semper,No,No,15962.0,Sanyo,28.0


In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_columns = utility_df.dtypes[utility_df.dtypes == "object"].index.tolist()

enc = OneHotEncoder(sparse_output=False)
enc_data = enc.fit_transform(utility_df[cat_columns])
enc_columns = enc.get_feature_names_out().tolist()

encode_df = pd.DataFrame(enc_data, columns=enc_columns)
#display(encode_df.head())

# now lets merge the into the application dataframe then drop original columns
utility_df = utility_df.merge(encode_df, left_index=True, right_index=True)
utility_df = utility_df.drop(columns=cat_columns)

print(utility_df.columns)

utility_df.head()

Index(['System_Size_AC', 'Storage_Size_kW_AC', 'Total_System_Cost',
       'Generator_Quantity', 'Utility_SDGE', 'Service_City_ALPINE',
       'Service_City_BONITA', 'Service_City_BONSALL',
       'Service_City_BORREGO SPRINGS', 'Service_City_BOULEVARD',
       ...
       'Generator_Manufacturer_Sharp', 'Generator_Manufacturer_Silfab',
       'Generator_Manufacturer_SolarWorld', 'Generator_Manufacturer_Solaria',
       'Generator_Manufacturer_SunEdison', 'Generator_Manufacturer_SunPower',
       'Generator_Manufacturer_Sunspark', 'Generator_Manufacturer_Tesla',
       'Generator_Manufacturer_Trina', 'Generator_Manufacturer_Yingli Energy'],
      dtype='object', length=154)


,System_Size_AC,Storage_Size_kW_AC,Total_System_Cost,Generator_Quantity,Utility_SDGE,Service_City_ALPINE,Service_City_BONITA,Service_City_BONSALL,Service_City_BORREGO SPRINGS,Service_City_BOULEVARD,...,Generator_Manufacturer_Sharp,Generator_Manufacturer_Silfab,Generator_Manufacturer_SolarWorld,Generator_Manufacturer_Solaria,Generator_Manufacturer_SunEdison,Generator_Manufacturer_SunPower,Generator_Manufacturer_Sunspark,Generator_Manufacturer_Tesla,Generator_Manufacturer_Trina,Generator_Manufacturer_Yingli Energy
0,3.676,0.0,21709.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.639,0.0,38000.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.760,0.0,18000.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.776,0.0,22000.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.320,0.0,15962.0,28.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = utility_df['Total_System_Cost']
X = utility_df.drop(columns='Total_System_Cost')

## Test the affect of using different random_state values has on the r2

In [57]:
%%time

best_state = -1;
best_r2 = 0
best_mae = 0

for i in range(500):
    print("Testing Random State", i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=i)

    # scale the data
    # Fit the StandardScaler
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)
    
    model = xgb.XGBRegressor(max_depth=10, n_estimators=100, learning_rate = 0.07, reg_alpha=1, subsample = 0.8, colsample_bytree=0.9)                     
    #model = GradientBoostingRegressor()
    #model = RandomForestRegressor(n_jobs = -1)
    #model = LGBMRegressor(metric='mae', n_estimators=50, num_leaves=256, bagging_freq=1, learning_rate = 0.07, reg_alpha=1, subsample = 0.8, colsample_bytree=0.5)
    model.fit(X_train, y_train)

    #Making predictions on the test set
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    print("Root Mean Squared Error:", rmse)
    print("Mean Absolute Error:", mae)
    print("R-squared Score:", r2)
    print()

    if r2 > best_r2:
        best_state = i
        best_r2 = r2
        best_mae = mae

# print out the best r2 so far
print("\n------------------------------")
print("Best Random State:", best_state)
print("Best Mean Absolute Error:", best_mae)
print("Best R-squared Score:", best_r2)

Testing Random State 0
Root Mean Squared Error: 19398.978834256948
Mean Absolute Error: 7205.7835888683285
R-squared Score: 0.4189553402409931

Testing Random State 1
Root Mean Squared Error: 19245.658767297078
Mean Absolute Error: 7183.715707927592
R-squared Score: 0.46505634288899167

Testing Random State 2
Root Mean Squared Error: 13962.743530001286
Mean Absolute Error: 7184.583482102666
R-squared Score: 0.7012900613563156

Testing Random State 3
Root Mean Squared Error: 18796.30802568715
Mean Absolute Error: 7248.020277834046
R-squared Score: 0.6022660994704845

Testing Random State 4
Root Mean Squared Error: 15586.609746162536
Mean Absolute Error: 7206.009343392695
R-squared Score: 0.5695917971171293

Testing Random State 5
Root Mean Squared Error: 20241.16881253389
Mean Absolute Error: 7338.3739172307205
R-squared Score: 0.609832764048027

Testing Random State 6
Root Mean Squared Error: 14353.522700842694
Mean Absolute Error: 7224.436012344699
R-squared Score: 0.7028759959599205


In [9]:
# Split the preprocessed data into a training and testing dataset
# split on 153 out of 200 for PGE
# split on 358 out of 500 for full dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=358)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# set the number of features
number_input_features = X_train.shape[1]
number_input_records = X_train.shape[0]

print("Number of Input Features:", number_input_features)
print("Number of Input Records:", number_input_records)

Number of Input Features: 153
Number of Input Records: 59934


In [48]:
#Basic model Creating an XGBoost regressor
#model = xgb.XGBRegressor(max_depth=10, n_estimators=100, learning_rate = 0.07, reg_alpha=1, subsample = 0.8, colsample_bytree=0.9)
                         
#model = RandomForestRegressor(n_jobs = -1)

#model = LGBMRegressor(metric='mae')
model = LGBMRegressor(metric='mae', n_estimators=500, num_leaves = 32, bagging_freq=1, learning_rate = 0.01, reg_alpha=1, subsample = 0.8, colsample_bytree=0.5)

#Training the model on the training data
model.fit(X_train_scaled, y_train)

#Making predictions on the test set
predictions = model.predict(X_test_scaled)

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012595 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 803
[LightGBM] [Info] Number of data points in the train set: 59934, number of used features: 123
[LightGBM] [Info] Start training from score 29527.443404
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [49]:
# Calculate the mean squared error and R-squared score
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared Score:", r2)

Root Mean Squared Error: 16443.233737232436
Mean Absolute Error: 7629.057531091966
R-squared Score: 0.5653477082964491


In [ ]:
# specify model and parameter range for grid search CV
# https://www.kaggle.com/code/jayatou/xgbregressor-with-gridsearchcv
# https://github.com/albertkklam/XGBRegressor/blob/master/XGBRegressor.ipynb
estimator = xgb.XGBRegressor(
    objective= 'reg:squarederror',
    colsample_bytree = 0.9,
    reg_alpha = 1,
    seed=42
)

parameters = {
    'max_depth': range (6, 12, 1),
    'n_estimators': range(50, 200, 10),
    'learning_rate': [0.1, 0.08, 0.06],
    'subsample': [0.7, 0.8, 0.9]
}

# specify the grid search object
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'r2',
    n_jobs = -1,
    cv = 5,
    verbose=True
)

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
# see the best parameters https://www.projectpro.io/recipes/find-optimal-parameters-using-gridsearchcv
print(" Results from Grid Search")
print("\n The best estimator across ALL searched params:\n", grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_search.best_params_)

In [ ]:
# Calculate the mean squared error and R-squared score
model = grid_search.best_estimator_
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared Score:", r2)

In [ ]:
# plot feature importance
n_top_features = 20
sorted_idx = model.feature_importances_.argsort()[::-1]

top_features = X_test.columns[sorted_idx][:n_top_features ].tolist()

plt.barh(X_test.columns[sorted_idx][:n_top_features ], model.feature_importances_[sorted_idx][:n_top_features ])

In [ ]:
# save the model for future use
model_file = "models/xgb_model-" + utility + ".json"
model.save_model(model_file)

## Test a Neural Network Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hidden_nodes_layer1 = number_input_features*2
hidden_nodes_layer2 = number_input_features

activation_function = 'relu'

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation_function))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

In [ ]:
# compile the model
nn.compile(loss='mae', optimizer='rmsprop', metrics=['mse','mae'])

In [ ]:
# Train the model
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

fit_model = nn.fit(X_train_scaled, y_train, epochs=500, batch_size=32, callbacks=[earlyStop])
#fit_model = nn.fit(X_train_scaled, y_train, epochs=100,  batch_size=16)

In [ ]:
#Making predictions on the test set
predictions = nn.predict(X_test_scaled)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, predictions)
#metric = tf.keras.metrics.R2Score()
#metric.update_state(y_test.to_numpy().reshape(-1,1), predictions)
#r2 = metric.result()

print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared Score:", r2)